# API - Boix Connect - Data Collect

<br>

Autor.: Sérgio C. Medina

In [2]:
# LIB
import json
import requests
import pandas as pd 

In [20]:
# Load Credentials Boix Connect
credentials = None
with open('../../secrets/boixconnect.json', 'r') as file:
    credentials = json.load(file)

del file

In [ ]:
## Get New Token
url = 'https://boixconnect.com/API_NEW_TOKEN/'
resp = requests.get(url, data=json.dumps(credentials))
token = None

if resp.status_code == 200:
    print(resp.text)
    msg = resp.json()
    print('msg', 'status:', msg['status'], 'message:', msg['message'])
    if msg['status'] == 200:
        token = msg['token']
    else:
        token = None
else:
    print('ERRO:', resp.status_code)
    token = None


In [62]:
## Get Users List
url = 'https://boixconnect.com/API_USERS/'
data = '{"token":"' + token + '"}' 
resp = requests.get(url, data=data)
if resp.status_code == 200:
    users = None
    try:
        users = resp.json()
    except:
        sResp = resp.text.replace('}{','},{')
        sResp = '['+sResp+']'
        users = json.loads(sResp)

    if users is not None:
        df_users = pd.json_normalize(users)
    else:
        df_users = None
        print('Response is empyt!')
    

else:
    print('ERRO:', resp.status_code)

del data, sResp


In [63]:
## Get Service Status
url = 'https://boixconnect.com/API_SERVICE_STATUS/'
data = '{"token":"' + token + '"}' 
resp = requests.get(url, data=data)
if resp.status_code == 200:
    service = None
    try:
        service = resp.json()
    except:
        sResp = resp.text.replace('}{','},{')
        sResp = '['+sResp+']'
        service = json.loads(sResp) 

    if service is not None:
        df_services = pd.json_normalize(service)
    else:
        df_services = None
        print('Response is empyt!')

else:
    print('ERRO:', resp.status_code)

In [ ]:
## Get Status VPN
url = 'https://boixconnect.com/API_STATUS_VPN/'
df_status = None

def getStatusVPN(id):
    data = '{"token":"' + token + '","machine":"' + id + '"}' 
    resp = requests.get(url, data=data)
    if resp.status_code == 200:
        vpn = None
        try:
            vpn = resp.json()
        except:
            sResp = resp.text.replace('}{','},{')
            sResp = '['+sResp+']'
            vpn = json.loads(sResp)

        if service is not None:
            df = pd.json_normalize(vpn)
        else:
            df = None
    else:
        print('ERRO:', resp.status_code)
        df = None

    return df


for index, row in df_services.iterrows():
    print(row['ID'], row['SERIALN'])

    df = getStatusVPN(id=row['ID'])
    if df_status is None:
        df_status = df.copy()
    else:
        df_status = df_status.append(df, ignore_index = True)




In [ ]:
## Get Info of Machines

url = 'https://boixconnect.com/API_LIST_MACHINES/'
df_machineinfo = None

def getMachineInfo(id):
    data = '{"token":"' + token + '","machine":"' + id + '"}' 
    resp = requests.get(url, data=data)
    if resp.status_code == 200:
        info = None
        try:
            info = resp.json()
        except:
            sResp = resp.text.replace('}{','},{')
            sResp = '['+sResp+']'
            info = json.loads(sResp)

        if service is not None:
            df = pd.json_normalize(info)
        else:
            df = None
    else:
        print('ERRO:', resp.status_code)
        df = None

    return df


for index, row in df_services.iterrows():
    print(row['ID'], row['SERIALN'])

    df = getMachineInfo(id=row['ID'])
    if df is not None:
        if df_machineinfo is None:
            df_machineinfo = df.copy()
        else:
            df_machineinfo = df_machineinfo.append(df, ignore_index = True)


In [115]:
## Get Machine Data
url = 'https://boixconnect.com/API_MACHINE_DATA/'
df_machinedata = None

def getMachineData(id, gid):
    data = '{"token":"' + token + '","machine":"' + id +'","group":"' + gid + '","lang":"2"}'
    resp = requests.get(url, data=data)
    if resp.status_code == 200:
        machine = None
        try:
            machine = resp.json()
        except:
            sResp = resp.text.replace('}{','},{')
            sResp = '['+sResp+']'
            machine = json.loads(sResp)

        if service is not None:
            try:
                df = pd.json_normalize(machine['VALUES'])
                df['DATE'] = machine['DATE']
                df['ID_MACHINE'] = machine['ID_MACHINE']
                df['SN'] = machine['SN']
                df['ID_GROUP'] = gid
                df['GROUP'] = machine['GROUP']
                df = df[['DATE','ID_MACHINE','SN','ID_GROUP','GROUP','DATO','VALOR']]
            except:
                print(machine)
                df = None
        else:
            df = None
    else:
        print('ERRO:', resp.status_code)
        df = None

    return df


for index, row in df_services.iterrows():
    print(row['ID'], row['SERIALN'])

    for i in range(1,14):
        df = getMachineData(id=row['ID'], gid=str(i))
        if df is not None:
            if df_machinedata is None:
                df_machinedata = df.copy()
            else:
                df_machinedata = df_machinedata.append(df, ignore_index = True)    


7 006/19
4 007/19
5 008/19
13 015/19
12 016/19
15 017/19
16 018/19
17 019/19
18 020/19
19 021/19
20 022/19
21 023/19
22 024/19
99 207/20
100 208/20
101 209/20
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
102 210/20
103 211/20
104 212/20
105 218/20
106 219/20
107 220/20
108 221/20
109 222/20
110 225/20
111 226/20
143 463/21
144 464/21
145 465/21
146 472/21
147 473/21
148 478/21
149 479/21
150 480/21
151 481/21


In [149]:
df_machinedata.loc[(df_machinedata.ID_MACHINE=='151') & (df_machinedata.ID_GROUP=='1')]

,DATE,ID_MACHINE,SN,ID_GROUP,GROUP,DATO,VALOR
3993,2021-06-22,151,481/21,1,Alarmas Variador,Encoder Fault Motor X Code 1,241
3994,2021-06-22,151,481/21,1,Alarmas Variador,Encoder Fault Motor X Code 2,241
3995,2021-06-22,151,481/21,1,Alarmas Variador,Encoder Fault Motor X Code 3,0
3996,2021-06-22,151,481/21,1,Alarmas Variador,Encoder Fault Motor X Code 4,0
3997,2021-06-22,151,481/21,1,Alarmas Variador,Encoder Fault Motor Y Code 1,241
3998,2021-06-22,151,481/21,1,Alarmas Variador,Encoder Fault Motor Y Code 2,241
3999,2021-06-22,151,481/21,1,Alarmas Variador,Encoder Fault Motor Y Code 3,0
4000,2021-06-22,151,481/21,1,Alarmas Variador,Encoder Fault Motor Y Code 4,0
